# Binary Model

In [ ]:
import pandas as pd
dsMeta = pd.read_csv('/kaggle/input/datasetfinals/train/train_meta.csv')
dsMeta

In [ ]:
dsMeta.info()

In [ ]:
dsMeta.set_index('record_name', inplace=True)

In [ ]:
dsTrue = pd.read_csv('/kaggle/input/datasetfinals/train/train_gts.csv')
dsTrue

In [ ]:
dsTrue.set_index('record_name', inplace=True)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np                      ## загрузка экг данных
train_X = []           
train_Y = []
for key in list(dsTrue.index):
  train_X.append(np.load('/kaggle/input/datasetfinals/train/'+key+'.npy'))
  per = dsMeta.loc[key]
  per['y'] = dsTrue.loc[key]['myocard']
  train_Y.append(per)

In [ ]:
from sklearn.preprocessing import StandardScaler            ## нормализование данных
import numpy as np


all_data = np.concatenate(train_X, axis=0)
print(all_data.shape)

scaler = StandardScaler()
scaler.fit(all_data)

normalized_data_list = [scaler.transform(data) for data in train_X]

train_X = normalized_data_list

In [ ]:
from sklearn.model_selection import train_test_split  ## деление данных

splitted_data = train_test_split(train_X, train_Y, test_size=0.1,random_state=48)
X_train_first, X_train_second, y_train_first, y_train_second = splitted_data

In [ ]:
# X_train_first = train_X                ## обучение на полном датасете
# y_train_first = train_Y

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
import torch.nn.functional as F
X = torch.from_numpy(np.array(X_train_first))
y = torch.from_numpy(np.array([i['y'] for i in y_train_first]))

In [ ]:
X_slices = []                                ## аугментирование данных: деление на кусочки по 800 измерений
y_slices = []
#cat_slices = []
#fullX_slices = []
step = 100
for i in range(0, 5000-800, step): #Скольжение окном с шагом 100 по 800 измерений
    X_slices.append(X[:,:,i:i+800])
    y_slices.append(y)

X_new = torch.cat(X_slices, dim=0)
y_new = torch.cat(y_slices, dim=0)
X_new.shape

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(X_new, y_new)

batch_size = 128

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
class ECG_CNN_TransformerBIN(nn.Module):
    def __init__(self):
        super(ECG_CNN_TransformerBIN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=12, out_channels=64, kernel_size=7, padding=3)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, padding=2)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1,)

        self.encoder_layer = nn.TransformerEncoderLayer(d_model=12, nhead=3)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=1)

        self.fc1 = nn.Linear(256*800 + 12*800 , 128)
        self.fc2 = nn.Linear(128, 1)
        self.relu = nn.ReLU()
        self.sig = nn.Sigmoid()
        self.drop = nn.Dropout(0.2)
        self.drop5 = nn.Dropout(0.8)

    def forward(self, x):
        cnn_x = self.relu(self.conv1(x))
        cnn_x = self.relu(self.conv2(cnn_x))
        cnn_x = self.relu(self.conv3(cnn_x))
        cnn_x = cnn_x.view(x.size(0), -1)

        #cnn_xf = self.relu(self.conv1(full_x))
        #cnn_xf = cnn_xf.view(full_x.size(0), -1)
        #cnn_xf = self.drop5(cnn_xf)

        x = x.permute(2, 0, 1)
        transformer_x = self.transformer_encoder(x)
        transformer_x = transformer_x.permute(1, 0, 2)
        transformer_x = transformer_x.reshape(transformer_x.size(0), -1)
        #transformer_x = self.drop(transformer_x) #drop показал себя хуже

        #Конкатинируем результаты cnn и transformer слоёв
        x = torch.cat((cnn_x, transformer_x), dim=1)

        x = self.relu(self.fc1(x))
        #x = self.drop(x)
        x = self.fc2(x)
        #x = self.sig(x)
        return x

In [ ]:
from tqdm.autonotebook import tqdm
import sklearn

In [ ]:
#del model
torch.cuda.empty_cache()
modelBIN = ECG_CNN_TransformerBIN().cuda()

#WithLogits показал себя лучше
loss_function = nn.BCEWithLogitsLoss()

optimizer = optim.AdamW(modelBIN.parameters(), lr=0.000001)
from torch.optim import lr_scheduler
#scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.0000003)

In [ ]:
def inference_ov(model, x, overlap=0.875):
    num_windows = int((x.shape[1] * overlap) // 800) + (1 if (x.shape[1] * overlap) % 800 != 0 else 0)
    step_size = int(800 * overlap)
    preds = []

    with torch.no_grad():
        model.eval()

        for i in range(num_windows):
            start = i * step_size
            end = start + 800
            x_window = x[:, start:end]
            pred = model(x_window.unsqueeze(0))
            preds.append(pred.item())

        avg_pred = sum(preds) / num_windows

    return avg_pred

In [ ]:
def add_noise(seq, target, noise_weight=0.05):
    noise = torch.randn_like(seq) * noise_weight

    seq_noisy = seq + noise

    seq = torch.cat((seq, seq_noisy), dim=0)
    target = torch.cat((target, target), dim=0)  

    return seq, target

In [ ]:
best_model = ''     ## обучение модели
f1Max = 0
from copy import deepcopy

loss_arr = []
epoch_qty = 100 #Кол-во эпох
for epoch in range(epoch_qty):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    for iter, (seq, target) in enumerate(tqdm(dataloader)):
        seq, target = add_noise(seq, target, 0.05) #Добавление шума

        y_pred = modelBIN(seq.cuda())

        loss = loss_function(y_pred.squeeze().cuda(), target.float().cuda())
        loss_arr.append(loss.item())
        loss.backward()
        optimizer.step()

#     eval_res = []                     ## evaluation модели если есть валидационная часть
#     for x in range(len(X_train_second)):
#        eval_res.append(inference_ov(model, torch.from_numpy(X_train_second[x]).cuda()))
#     f1 = sklearn.metrics.f1_score([i['y'] for i in y_train_second],[1 if float(i) >= 0.275 else 0 for i in eval_res])
#     print(f1)
#     if f1 > f1Max:
#         best_model = deepcopy(model.state_dict())
#         f1Max = f1
      
    
    print(sum(loss_arr)/len(loss_arr))
    if sum(loss_arr)/len(loss_arr) <= 0.03: #При данном значении результат на паблике и вале был наилучшим
      break
    loss_arr = []
    if (epoch+1) % 10 == 0:
      print(f'Epoch: {epoch}, Loss: {str(loss.item())}')

In [ ]:
#best_modelBIN = deepcopy(best_model)
# torch.save(modelBIN.state_dict(), '3heads1layerONFULLSET0.02lossBIN.pt')

# 6 Classes Model

Данные открываются сначала и некоторые библиотеки импортируются заново, так как обучение 2х моделей производилось и по отдельности

In [ ]:
import pandas as pd
dsMeta = pd.read_csv('/kaggle/input/datasetfinals/train/train_meta.csv')
dsMeta

In [ ]:
dsMeta.set_index('record_name', inplace = True)

In [ ]:
import pandas as pd
dsTrue = pd.read_csv('/kaggle/input/datasetfinals/train/train_gts_final.csv')
dsTrue

In [ ]:
dsTrue.set_index('record_name', inplace=True)

In [ ]:
import numpy as np
from copy import copy
train_X = []
train_Y = []
for key in list(dsTrue.index):
  train_X.append(np.load('/kaggle/input/datasetfinals/train/'+key+'.npy'))
  per = dsMeta.loc[key].copy()
#   print(list(dsTrue.loc[key]))
  per['y'] = list(dsTrue.loc[key])
  train_Y.append(per['y'])

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np

all_data = np.concatenate(train_X, axis=0)
print(all_data.shape)

scaler = StandardScaler()
scaler.fit(all_data)

normalized_data_list = [scaler.transform(data) for data in train_X]

train_X = normalized_data_list

In [ ]:
from sklearn.model_selection import train_test_split


splitted_data = train_test_split(train_X, train_Y, test_size=0.1,random_state=48)
X_train_first, X_train_second, y_train_first, y_train_second = splitted_data

In [ ]:
# X_train_first = train_X      ##обучение на полном датасете
# y_train_first = train_Y

In [ ]:
import torch.nn.functional as F
X = torch.from_numpy(np.array(X_train_first))
y = torch.from_numpy(np.array(y_train_first))

In [ ]:
# from tqdm import tqdm
# def add_noise_to_sample(sample, noise_level=0.001):
#     noise = torch.randn_like(sample) * noise_level
#     return sample + noise

# lengthh = X.shape[0]                         ##Применение заранее шума к обычным данным не дало прироста F1
# new_X_balanced = []
# new_y_balanced = []
# for xx in tqdm(range(lengthh)):
# #     print(xx)
#     sample_X_balanced = X[xx]
#     new_X_balanced.append(add_noise_to_sample(sample_X_balanced))
#     new_X_balanced.append(sample_X_balanced)
    
#     new_y_balanced.append(y[xx])
#     new_y_balanced.append(y[xx])
    
# X = torch.stack(new_X_balanced, dim = 0)
# y = torch.stack(new_y_balanced, dim = 0)

In [ ]:
X_slices = []
y_slices = []

step = 50
for i in range(0, 5000-700, step): 
    X_slices.append(X[:,:,i:i+700])
    y_slices.append(y)

X_new = torch.cat(X_slices, dim=0)  
y_new = torch.cat(y_slices, dim=0) 

In [ ]:
sum_first_6_classes = 0#sum(counts[:-1])  # используя вашу переменную counts

# Индексы для каждого класса
indices = [torch.where(y_new[:,i]==1)[0] for i in range(y_new.shape[1])]

# Получаем случайную перестановку индексов для последнего класса и берем нужное количество
random_indices_last_class = indices[-1][torch.randperm(len(indices[-1]))[:sum_first_6_classes]]

# Индексы для нового массива данных
new_indices = torch.cat(indices[:-1] + [random_indices_last_class])

# Новые данные
X_balanced = X_new[new_indices]
y_balanced = y_new[new_indices]

In [ ]:
y_balanced = torch.tensor([[int(b) for b in i[:6]] for i in y_balanced])

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(X_balanced, y_balanced)

batch_size = 128

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
X2 = []
Y2 = []

for i in range(len(X_train_second)):
    if y_train_second[i][-1] != 1:
        X2.append(X_train_second[i])
        Y2.append(y_train_second[i][:6])

In [ ]:
def inference_ov(model, x, overlap=0.875):
    num_windows = int((x.shape[1] * overlap) // 700) + (1 if (x.shape[1] * overlap) % 700 != 0 else 0)
    step_size = int(700 * overlap)
    preds = []
    
    with torch.no_grad():
        model.eval()
        
        for i in range(num_windows):
            start = i * step_size
            end = start + 700
            x_window = x[:, start:end]
            pred = model(x_window.unsqueeze(0))
            preds.append(pred)
        
        avg_pred = sum(preds) / num_windows
        
    return avg_pred

In [ ]:
import math
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
class ECG_CNN_Transformer(nn.Module):
    def __init__(self):
        super(ECG_CNN_Transformer, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=12, out_channels=64, kernel_size=7, padding=3)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, padding=2)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1,)
        
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=12, nhead=2)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=1)
        #+ 64*1000
        self.fc1 = nn.Linear(256*700 + 12*700 , 128) 
        self.fc2 = nn.Linear(128, 6)
        self.relu = nn.ReLU()
        self.sig = nn.Sigmoid()
        self.drop = nn.Dropout(0.2)
        self.drop5 = nn.Dropout(0.8)

    def forward(self, x):
        cnn_x = self.relu(self.conv1(x))
        cnn_x = self.relu(self.conv2(cnn_x))
        cnn_x = self.relu(self.conv3(cnn_x))
        cnn_x = cnn_x.view(x.size(0), -1) 

        x = x.permute(2, 0, 1)  
        transformer_x = self.transformer_encoder(x)
        transformer_x = transformer_x.permute(1, 0, 2) 
        transformer_x = transformer_x.reshape(transformer_x.size(0), -1)  
        #transformer_x = self.drop(transformer_x)

        # Concatenate CNN and Transformer outputs
        x = torch.cat((cnn_x, transformer_x), dim=1)

        x = self.relu(self.fc1(x))
        #x = self.drop(x)
        x = self.fc2(x)
        #x = self.sig(x)
        return x

In [ ]:
from tqdm.autonotebook import tqdm
import sklearn

In [ ]:
# del model
torch.cuda.empty_cache()
model = ECG_CNN_Transformer().cuda()
optimizer = optim.AdamW(model.parameters(), lr=0.000001)
from torch.optim import lr_scheduler

loss_function = nn.BCEWithLogitsLoss()

In [ ]:
# checkpoint16102023 = torch.load('/kaggle/input/fullf10-54loss0-14classes6/FULLf10.54loss0.14classes6.pt')
# model = ECG_CNN_Transformer().cuda()
# model.load_state_dict(checkpoint16102023.state_dict())

In [ ]:
def add_noise(seq, target, noise_weight=0.05):
    noise = torch.randn_like(seq) * noise_weight

    seq_noisy = seq + noise

    seq = torch.cat((seq, seq_noisy), dim=0)
    target = torch.cat((target, target), dim=0)  

    return seq, target

In [ ]:
f1Max = 0
bestModel = ''
from copy import deepcopy
loss_arr = []
for epoch in range(200):
    
    model.train(True)
    
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    for iter, (seq, target) in enumerate(tqdm(dataloader)):
        seq, target = add_noise(seq, target, 0.05) #Добавление шума
        
        y_pred = model(seq.cuda())
        loss = loss_function(y_pred.cuda(), target.float().cuda())
        loss.backward()
        
        optimizer.step()

        loss_arr.append(loss.item())


#     model.eval()
#     eval_res = []
#     for x in range(len(X2)):
#        eval_res.append(inference_ov(model, torch.from_numpy(X2[x]).cuda()))
# #            print(eval_res)
#     new_arr = []
#     for k in range(len(eval_res)):
#         for j in range(len(eval_res[k])):
#             new_arr.append(eval_res[k][j].detach().cpu().numpy())
#     new_arr = [[sigmoid(j) for j in i] for i in new_arr]
#     f1 = sklearn.metrics.f1_score(Y2,[[1 if float(j) >= 0.20 else 0 for j in i]for i in new_arr], average = 'macro')
#     print(f"F1_score = {f1}")
#     if f1 > f1Max:
#         f1Max = f1
#         bestModel = deepcopy(model.state_dict())
#     print(f"F1_score = {f1}")

    

    print(sum(loss_arr)/len(loss_arr))
    
    if sum(loss_arr)/len(loss_arr) <= 0.15: #При данном значении результат был наилучшим
      break
    loss_arr = []

    print(f'Epoch: {epoch}, Loss: {str(loss.item())}')

In [ ]:
model.eval()       ## evalutation модели и перебор лучшего трешхолда
eval_res = []
for x in range(len(X2)):
   eval_res.append(inference_ov(model, torch.from_numpy(X2[x]).cuda()))

new_arr = []
for k in range(len(eval_res)):
    for j in range(len(eval_res[k])):
        new_arr.append(eval_res[k][j].detach().cpu().numpy())
new_arr = [[sigmoid(j) for j in i] for i in new_arr]
sklearn.metrics.f1_score(Y2,[[1 if float(j) >= 0.1 else 0 for j in i]for i in new_arr], average = 'macro')

In [ ]:
# torch.save(model.state_dict(), 'classes6heads2ONFULLSET0.12loss.pt')

# Работа с тестовыми данными и создание сабмита

In [ ]:
import pandas as pd
dsMetatest = pd.read_csv('/kaggle/input/datasetfinalstest/test/test_meta.csv')
dsMetatest

In [ ]:
record_names = list(dsMetatest['record_name'])
from copy import copy
y_train_secondtest = copy(dsMetatest)
dsMetatest.set_index('record_name', inplace=True)

In [ ]:
import numpy as np
test_X = []
test_meta = []

for key in record_names:
  test_X.append(np.load('/kaggle/input/datasetfinalstest/test/'+key+'.npy'))
  test_meta.append(dsMetatest.loc[key])

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np

normalized_data_listtest = [scaler.transform(data) for data in test_X]

test_X = normalized_data_listtest

In [ ]:
dsMetaTest = pd.read_csv('/kaggle/input/datasetfinalstest/test/test_meta.csv')
train_record_names = dsMetaTest['record_name']

In [ ]:
def inference_ov(model, x, overlap=0.875):
    num_windows = int((x.shape[1] * overlap) // 800) + (1 if (x.shape[1] * overlap) % 800 != 0 else 0)
    step_size = int(800 * overlap)
    preds = []
        
    with torch.no_grad():
        model.eval()
        
        for i in range(num_windows):
            start = i * step_size
            end = start + 800
            x_window = x[:, start:end]
            pred = model(x_window.unsqueeze(0))
            preds.append(pred)
        
        avg_pred = sum(preds) / num_windows
        
    return avg_pred

In [ ]:
# modelBIN = ECG_CNN_TransformerBIN().cuda()
# modelBIN.load_state_dict(torch.load('/kaggle/input/balancedbin0-09loss2heads1layer/balancedBIN0.09loss2heads1layer.pt').state_dict())

In [ ]:
eval_res = []
for x in range(len(test_X)):
   eval_res.append(inference_ov(modelBIN, torch.from_numpy(test_X[x]).cuda()))
eval_res = [sigmoid(i) for i in eval_res]
# print(sklearn.metrics.f1_score([i['y'] for i in y_train_second],[1 if float(i) >= 0.325 else 0 for i in eval_res]))

In [ ]:
def inference_ov(model, x, overlap=0.875):
    num_windows = int((x.shape[1] * overlap) // 700) + (1 if (x.shape[1] * overlap) % 700 != 0 else 0)
    step_size = int(700 * overlap)
    preds = []
    
    with torch.no_grad():
        model.eval()
        
        for i in range(num_windows):
            start = i * step_size
            end = start + 700
            x_window = x[:, start:end]
            pred = model(x_window.unsqueeze(0))
            preds.append(pred)
        
        avg_pred = sum(preds) / num_windows
        
    return avg_pred

In [ ]:
# checkpoint = torch.load('/kaggle/input/classes6heads2onfullset0-085loss/classes6heads2ONFULLSET0.085loss.pt')
# model  = ECG_CNN_Transformer().cuda()
# model.load_state_dict(checkpoint)

In [ ]:
answers = []
for i in range(len(y_train_secondtest)):
  test_record = np.load('/kaggle/input/datasetfinalstest/test/'+record_names[i]+'.npy')
  test_record = scaler.transform(test_record)

  answers.append(inference_ov(model, torch.from_numpy(test_record).cuda()))

In [ ]:
reses = [1 if float(j) >= 0.275 else 0 for j in list(eval_res)]

In [ ]:
import pandas as pd
dsTrue = pd.read_csv('/kaggle/input/datasetfinals/train/train_gts_final.csv')
dsTrue

In [ ]:
dsTrue.set_index('record_name', inplace=True)

In [ ]:
import math
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [ ]:
new_arr = []
for k in range(len(answers)):
    for j in range(len(answers[k])):
        new_arr.append(answers[k][j].detach().cpu().numpy())

for i in range(len(new_arr)):
    for j in range(len(new_arr[i])):
        try:
            new_arr[i][j] = sigmoid(new_arr[i][j])
        except:
            new_arr[i][j] = 0

ress = []
for kk in range(len(reses)):
    if reses[kk] == 0:
        ress.append([0,0,0,0,0,0,1])
    else:
#         tmp_ress = [1 if float(j) >= 0.1 else 0 for j in new_arr[kk]]+[0]
#         count_zero = 0
#         for ll in tmp_ress:
#             if ll == 0:
#                 count_zero += 1            использование максимального элемента вместо трешхолда [1, 2, 3] -> [0, 0, 1]
#         if count_zero == 7:                не привело к хорошим результатам
#             print(new_arr[kk].shape)
#             zero_mat = np.zeros(new_arr[kk].shape[0] + 1)
#             zero_mat[np.argmax(new_arr[kk], axis = 0)] = 1
# #             print(zero_mat)
# #             print(new_arr[kk])
# #             print(kk)
# #             break
#             ress.append(zero_mat.astype(int))
#             print(kk)
#         else:

        ress.append([1 if float(j) >= 0.1 else 0 for j in new_arr[kk]]+[0])
ans = pd.DataFrame(ress, columns = list(dsTrue.columns))

In [ ]:
ans['record_name'] = record_names
ans.set_index('record_name', inplace = True)

In [ ]:
sample = pd.read_csv('/kaggle/input/sample/sample_submission (5).csv')

In [ ]:
for i in range(len(sample)):
    sample.loc[i, 'перегородочный'] = ans.loc[sample.iloc[i]['record_name']]['перегородочный'] 
    sample.loc[i, 'передний'] = ans.loc[sample.iloc[i]['record_name']]['передний']
    sample.loc[i, 'боковой'] = ans.loc[sample.iloc[i]['record_name']]['боковой'] 
    sample.loc[i, 'передне-боковой'] = ans.loc[sample.iloc[i]['record_name']]['передне-боковой'] 
    sample.loc[i, 'передне-перегородочный'] = ans.loc[sample.iloc[i]['record_name']]['передне-перегородочный'] 
    sample.loc[i, 'нижний'] = ans.loc[sample.iloc[i]['record_name']]['нижний'] 
    sample.loc[i, 'норма'] = ans.loc[sample.iloc[i]['record_name']]['норма'] 

In [ ]:
# sample.to_csv('bestansesBIN+FULLf10.54loss0.05classes6Threshold01andOV0875.csv', index = False)

**ИСТОРИЯ РАБОТЫ (СКОПИРОВАНЫ ИЗ ДРУГИХ БЛОКНОТОВ)**

In [ ]:
###АЛГОРИТМ ПЕРЕБОРА ЛУЧШЕГО ТРЕШХОЛДА

from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
import numpy as np

predictions = [preds_class[i][1]*0.8 + 0.4*sigmoid(preds_graph[i]) for i in range(len(preds_class))]
labels = train_Y

from sklearn.metrics import f1_score

def find_best_threshold(y_true, y_proba):
    thresholds = np.arange(0.0, 1.0, 0.001)
    best_f1 = 0
    best_threshold = 0

    for threshold in thresholds:
        y_pred = [1 if prob >= threshold else 0 for prob in y_proba]
        f1 = f1_score(y_true, y_pred)

        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
    print(best_f1)
    return best_threshold

best_threshold = find_best_threshold(labels, predictions)
print(best_threshold)

In [ ]:
import torch
import torch.nn as nn

class F1Loss(nn.Module):
    def __init__(self, epsilon=1e-7):
        super().__init__()
        self.epsilon = epsilon
        
    def forward(self, y_pred, y_true):
        # Преобразование предсказаний в бинарные значения
        y_pred = torch.round(torch.sigmoid(y_pred))  # добавляем sigmoid, если не использовали его ранее
        
        # Вычисление TP, FP, FN
        tp = torch.sum(y_true * y_pred, dim=1)
        fp = torch.sum((1 - y_true) * y_pred, dim=1)
        fn = torch.sum(y_true * (1 - y_pred), dim=1)

        # Вычисление precision и recall
        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        # Вычисление F1 Score
        f1_score = 2 * (precision * recall) / (precision + recall + self.epsilon)

        # Возвращение F1 loss: 1 - среднее значение F1 Score
        return 1 - torch.mean(f1_score)

**ИСПРОБОВАННЫЕ ВАРИАНТЫ pytorch моделей**

In [ ]:
class ECG_CNN_Transformer(nn.Module):
    def __init__(self):
        super(ECG_CNN_Transformer, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=12, out_channels=64, kernel_size=7, padding=3)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, padding=2)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1,)
        
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=12, nhead=3)  ### (1 - 3)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2) ### (1 - 2)
        #+ 64*1000
        self.fc1 = nn.Linear(256*700 + 12*700 , 128) # Concatenating output of CNN and Transformer
        self.fc2 = nn.Linear(128, 6)
        self.relu = nn.ReLU()
        self.sig = nn.Sigmoid()
        self.drop = nn.Dropout(0.2)
        self.drop5 = nn.Dropout(0.8)

    def forward(self, x):
        cnn_x = self.relu(self.conv1(x))
        cnn_x = self.relu(self.conv2(cnn_x))
        cnn_x = self.relu(self.conv3(cnn_x))
        cnn_x = cnn_x.view(x.size(0), -1) # flatten the tensor
        
        #cnn_xf = self.relu(self.conv1(full_x))
        #cnn_xf = cnn_xf.view(full_x.size(0), -1) # flatten the tensor
        #cnn_xf = self.drop5(cnn_xf)

        x = x.permute(2, 0, 1)  # Transformer expects input as (seq_len, batch, features)
        transformer_x = self.transformer_encoder(x)
        transformer_x = transformer_x.permute(1, 0, 2)  # Back to (batch, seq_len, features)
        transformer_x = transformer_x.reshape(transformer_x.size(0), -1)  # flatten the tensor
        #transformer_x = self.drop(transformer_x)

        # Concatenate CNN and Transformer outputs
        x = torch.cat((cnn_x, transformer_x), dim=1)

        x = self.relu(self.fc1(x))
        #x = self.drop(x)
        x = self.fc2(x)
        #x = self.sig(x)
        return x

In [ ]:
class AggregatingLSTMClassifier(nn.Module):
    def __init__(self, input_dim=12, hidden_dim=5, output_dim=1, num_layers=5):
        super().__init__()

        self.lstm = nn.Transformer(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(input_dim, output_dim)
        self.sig = nn.Sigmoid()

    def forward(self, x):

        lstm_out, _ = self.lstm(x.permute(0, 1, 2))


        out = self.fc(lstm_out)
        out = self.sig(out).mean(dim=1)

        return out.squeeze()

In [ ]:
class TransformerClassifier(nn.Module):
    def __init__(self, input_dim=12, hidden_dim=5, output_dim=1, num_layers=5):
        super().__init__()

        self.transformer = nn.TransformerEncoderLayer(input_dim, num_layers, dropout=0.1)
        self.fc = nn.Linear(input_dim, output_dim)
        self.fc2 = nn.Linear(5000, 1)
        self.sig = nn.Sigmoid()
        self.dp = nn.Dropout(0.2)

    def forward(self, x):
        transformer_out = self.transformer(x.permute(0, 2, 1))

        out = self.fc(transformer_out)

        out = self.fc2(out.permute(0,2,1))
        out = self.sig(out)
        return out.squeeze()

In [ ]:
class AggregatingEmbedLSTMClassifier(nn.Module):
    def __init__(self, input_dim=12, hidden_dim=5, output_dim=1, num_layers=5):
        super().__init__()
        self.embedding = nn.Embedding(5000,32)
        self.lstm = nn.LSTM(32, 5, 5, batch_first=True,bidirectional = False)
        self.fc = nn.Linear(input_dim, output_dim)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        embed_out = self.embedding(x.permute(0, 1, 2))
        lstm_out, _ = self.lstm(embed_out)


        out = self.fc(lstm_out)
        out = self.sig(out).mean(dim=1)

        return out.squeeze()

In [ ]:
import torch.nn as nn

class ECG_CNN(nn.Module):
    def __init__(self):
        super(ECG_CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=12, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1,)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1,)
        self.fc1 = nn.Linear(256*800, 128)
        self.fc2 = nn.Linear(128, 1)
        self.relu = nn.ReLU()
        self.sig = nn.Sigmoid()
        self.drop = nn.Dropout(0.2)


    def forward(self, x):
        x = self.relu(self.conv1(x))
        #x = self.drop(x)
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        x = self.sig(x)
        return x

In [ ]:
import torch.nn as nn

class ECG_CNN_slice(nn.Module):
    def __init__(self):
        super(ECG_CNN_slice, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=12, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1,)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1,)
        self.fc1 = nn.Linear(256*850, 128)
        self.fc2 = nn.Linear(128, 1)
        self.relu = nn.ReLU()
        self.sig = nn.Sigmoid()
        self.drop = nn.Dropout(0.2)

    def forward(self, x):
        x = self.slice(x)
        # print(x.shape)
        # print(1/0)
        x = self.relu(self.conv1(x))
        #x = self.drop(x)
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        x = self.sig(x)
        return x

In [ ]:
import torch.nn as nn

class ECG_Transformer(nn.Module):
    def __init__(self):
        super(ECG_Transformer, self).__init__()
        self.embedding = nn.Linear(12, 64)
        encoder_layers = nn.TransformerEncoderLayer(d_model=64, nhead=8)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=3)
        self.fc = nn.Linear(64, 1)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        x = x.permute(2, 0, 1)
        x = self.embedding(x)
        x = self.transformer_encoder(x)
        x = x.permute(1, 2, 0)
        x = torch.mean(x, dim=2)
        x = self.fc(x)
        x = self.sig(x)
        return x